In [1]:
# Importing the Violations Dataset with only required columns
import pandas as pd

df = pd.read_csv(r"Parking_Violations_Issued_2019-2021.csv",
                usecols = ['Summons Number','Plate ID','Registration State','Plate Type','Issue Date','Violation Code','Vehicle Body Type','Vehicle Make','Issuing Agency','Violation Precinct','Issuer Precinct','Violation Time','Violation County','Vehicle Color','Vehicle Year'])
df.shape

(23497573, 15)

In [2]:
df.head()

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Violation Precinct,Issuer Precinct,Violation Time,Violation County,Vehicle Color,Vehicle Year
0,1465574475,JMJ4156,99,PAS,01/01/2019 12:00:00 AM,98,SDN,HONDA,P,75,75,1048A,K,GOLD,0
1,1452758967,T753575C,NY,OMT,01/01/2019 12:00:00 AM,71,SDN,TOYOT,P,102,402,0150P,Q,BLK,2016
2,1460097804,JMU9251,NY,PAS,01/01/2019 12:00:00 AM,70,SDN,HONDA,P,113,113,0700A,Q,GY,2010
3,1452790565,AV32960,CT,999,01/01/2019 12:00:00 AM,40,SUBN,MITSU,P,115,401,1222P,Q,WHT,0
4,1455416400,2XU096,OK,PAS,01/01/2019 12:00:00 AM,17,DELV,INTER,P,1,105,0815A,NaN,WH,0


In [3]:
# Formatting the issue date timestamp and parsing the month-year
df['Issue Date'] = pd.to_datetime(df['Issue Date'])
df['Issue_month_year'] = df['Issue Date'].dt.strftime('%B-%Y')
df['Issue_year'] = df['Issue Date'].dt.strftime('%Y')

In [4]:
#Summarizing by month-year. Aggregation should be done at yearly level as monthly distributions are skewed.
df.Issue_month_year.value_counts(ascending = True)

March-2019             47
February-2019          68
April-2019            101
March-2021            165
February-2021         171
April-2020            186
April-2021            190
February-2020         195
May-2021              335
March-2020            374
January-2019          456
January-2020          565
January-2021          658
May-2019             8237
May-2020            12065
June-2019          226351
June-2020          240540
June-2021          251427
December-2019      992954
July-2020         1050234
December-2020     1161109
December-2021     1164148
July-2019         1164693
September-2021    1190870
November-2019     1191674
September-2019    1229541
November-2020     1264936
October-2019      1276499
November-2021     1284437
August-2019       1331903
October-2021      1346703
August-2020       1403657
July-2021         1404374
August-2021       1419732
October-2020      1421643
September-2020    1456335
Name: Issue_month_year, dtype: int64

In [5]:
# Checking for Valid State Codes
df['Registration State'].value_counts(ascending = True)

NT           3
YT           4
MX          13
SK          21
PE          26
        ...   
CT      344169
FL      425771
PA      716243
NJ     2219741
NY    17753475
Name: Registration State, Length: 68, dtype: int64

In [6]:
# Removing Invalid State Codes
state_codes = pd.read_excel(r"State-Codes.xlsx")
df = pd.merge(df, state_codes, left_on = "Registration State", right_on ='State Code' , how = "inner")
df.shape

(23415023, 20)

In [7]:
df.head()

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Violation Precinct,Issuer Precinct,Violation Time,Violation County,Vehicle Color,Vehicle Year,Issue_month_year,Issue_year,State,Abbreviation,State Code
0,1452758967,T753575C,NY,OMT,2019-01-01,71,SDN,TOYOT,P,102,402,0150P,Q,BLK,2016,January-2019,2019,New York,N.Y.,NY
1,1460097804,JMU9251,NY,PAS,2019-01-01,70,SDN,HONDA,P,113,113,0700A,Q,GY,2010,January-2019,2019,New York,N.Y.,NY
2,1464592822,31328WG,NY,COM,2019-01-02,14,DELV,FRUEH,P,18,401,0432P,NY,WHITE,2004,January-2019,2019,New York,N.Y.,NY
3,1464592810,63422JM,NY,COM,2019-01-02,14,DELV,INTER,P,18,401,0548P,NY,BROWN,1998,January-2019,2019,New York,N.Y.,NY
4,1464592809,48799JU,NY,COM,2019-01-02,14,VAN,DODGE,P,18,401,0555P,NY,WHITE,2005,January-2019,2019,New York,N.Y.,NY


In [8]:
#Checking for null values
print(df.isnull().sum())

Summons Number              0
Plate ID                  215
Registration State          0
Plate Type                  0
Issue Date                  0
Violation Code              0
Vehicle Body Type       65720
Vehicle Make            60392
Issuing Agency              0
Violation Precinct          0
Issuer Precinct             0
Violation Time            393
Violation County        58349
Vehicle Color         1151666
Vehicle Year                0
Issue_month_year            0
Issue_year                  0
State                       0
Abbreviation            94232
State Code                  0
dtype: int64


In [9]:
# Dropping the rows having NaN/NaT values in key attributes

df = df.dropna(subset=['Plate ID'])
df = df.dropna(subset=['Violation Time'])
 
# Resetting the indices using df.reset_index()
df = df.reset_index(drop=True)
df.shape

(23414415, 20)

In [10]:
#Imputing the missing values
df['Vehicle Body Type'].fillna(df['Vehicle Body Type'].mode()[0], inplace = True)
df['Vehicle Make'].fillna(df['Vehicle Make'].mode()[0], inplace = True)
df['Violation County'].fillna(df['Violation County'].mode()[0], inplace = True)
df['Vehicle Color'].fillna(df['Vehicle Color'].mode()[0], inplace = True)
df['Vehicle Year'].fillna(df['Vehicle Year'].mode()[0], inplace = True)

In [11]:
#Keeping Required columns
col_list = ['Summons Number','Plate ID','Registration State','Plate Type','Issue Date','Violation Code','Vehicle Body Type','Vehicle Make','Issuing Agency','Violation Precinct','Issuer Precinct','Violation Time','Violation County','Vehicle Color','Vehicle Year','State','Issue_month_year','Issue_year']
df = df[col_list]

In [12]:
print(df.isnull().sum())

Summons Number        0
Plate ID              0
Registration State    0
Plate Type            0
Issue Date            0
Violation Code        0
Vehicle Body Type     0
Vehicle Make          0
Issuing Agency        0
Violation Precinct    0
Issuer Precinct       0
Violation Time        0
Violation County      0
Vehicle Color         0
Vehicle Year          0
State                 0
Issue_month_year      0
Issue_year            0
dtype: int64


In [13]:
#Parsing the Violiaton Time
import datetime
import re
#tme='0612A' 
#pd.to_datetime(tme).strftime('%H:%M%p')
#df['Violation Time'] = pd.to_datetime(df['Violation Time'],format = '%H%m%p',errors='raise' )
#format = '%H%M%p'

df['new_violation_time'] = df['Violation Time'].str.replace(r'(\w{2})(\w{2})(\w{1})', r'\1:\2', regex=True)
df['new_violation_time_AP'] = df['Violation Time'].str.replace(r'(\w{2})(\w{2})(\w{1})', r'\3', regex=True)
df['new_violation_time_AP'] = df['new_violation_time'] + df['new_violation_time_AP'].apply(lambda n: 'AM' if n == 'A' else 'PM')
df.head()

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Violation Precinct,Issuer Precinct,Violation Time,Violation County,Vehicle Color,Vehicle Year,State,Issue_month_year,Issue_year,new_violation_time,new_violation_time_AP
0,1452758967,T753575C,NY,OMT,2019-01-01,71,SDN,TOYOT,P,102,402,0150P,Q,BLK,2016,New York,January-2019,2019,01:50,01:50PM
1,1460097804,JMU9251,NY,PAS,2019-01-01,70,SDN,HONDA,P,113,113,0700A,Q,GY,2010,New York,January-2019,2019,07:00,07:00AM
2,1464592822,31328WG,NY,COM,2019-01-02,14,DELV,FRUEH,P,18,401,0432P,NY,WHITE,2004,New York,January-2019,2019,04:32,04:32PM
3,1464592810,63422JM,NY,COM,2019-01-02,14,DELV,INTER,P,18,401,0548P,NY,BROWN,1998,New York,January-2019,2019,05:48,05:48PM
4,1464592809,48799JU,NY,COM,2019-01-02,14,VAN,DODGE,P,18,401,0555P,NY,WHITE,2005,New York,January-2019,2019,05:55,05:55PM


In [14]:
# Importing the metadata for registractin codes description, violation code description and vehicel body descrption
registration_codes = pd.read_excel(r"Registration-Codes-Metadata.xlsx")
violation_codes = pd.read_excel(r"ParkingViolationCodes_January2020.xlsx")
vehiclebody_codes = pd.read_excel(r"Vehicle-Body-Type-Metadata.xlsx")
registration_codes.head()


,Description,Code
0,Air National Guard,ARG
1,Army National Guard,AYG
2,County Board of Supervisors,CBS
3,County Clerk,CCK
4,County Legislators,CLG


In [15]:
# Merging the metadata for registractin codes description, violation code description and vehicel body descrption
df_1 = pd.merge(df, registration_codes, left_on = "Plate Type", right_on ='Code' , how = "left")
df_2 = pd.merge(df_1, violation_codes, left_on = "Violation Code", right_on ='VIOLATION CODE' , how = "left")
df_3 = pd.merge(df_2, vehiclebody_codes, left_on = "Vehicle Body Type", right_on ='Code' , how = "left")
df_3.head()

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Violation Precinct,...,new_violation_time,new_violation_time_AP,Description,Code_x,VIOLATION CODE,VIOLATION DESCRIPTION,Fine Amount,Fine_Amount_Others,Vehicle Description,Code_y
0,1452758967,T753575C,NY,OMT,2019-01-01,71,SDN,TOYOT,P,102,...,01:50,01:50PM,Taxi & Limousine Commission,OMT,71.0,INSP. STICKER-EXPIRED/MISSING,65.0,65.0,NaN,NaN
1,1460097804,JMU9251,NY,PAS,2019-01-01,70,SDN,HONDA,P,113,...,07:00,07:00AM,Passenger,PAS,70.0,REG. STICKER-EXPIRED/MISSING,65.0,65.0,NaN,NaN
2,1464592822,31328WG,NY,COM,2019-01-02,14,DELV,FRUEH,P,18,...,04:32,04:32PM,Commercial Vehicles,COM,14.0,NO STANDING-DAY/TIME LIMITS,115.0,115.0,NaN,NaN
3,1464592810,63422JM,NY,COM,2019-01-02,14,DELV,INTER,P,18,...,05:48,05:48PM,Commercial Vehicles,COM,14.0,NO STANDING-DAY/TIME LIMITS,115.0,115.0,NaN,NaN
4,1464592809,48799JU,NY,COM,2019-01-02,14,VAN,DODGE,P,18,...,05:55,05:55PM,Commercial Vehicles,COM,14.0,NO STANDING-DAY/TIME LIMITS,115.0,115.0,NaN,NaN


In [16]:
col_list = ['Summons Number','Plate ID','Registration State','Plate Type','Issue Date','Violation Code','Vehicle Body Type','Vehicle Make','Issuing Agency','Violation Precinct','Issuer Precinct','Violation Time','Violation County','Vehicle Color','Vehicle Year','State','Issue_month_year','Issue_year','new_violation_time_AP','Description','VIOLATION DESCRIPTION','Fine Amount']
final_df = df_3[col_list]
final_df.shape

(23426275, 22)

In [17]:
final_df.head()

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Violation Precinct,...,Violation County,Vehicle Color,Vehicle Year,State,Issue_month_year,Issue_year,new_violation_time_AP,Description,VIOLATION DESCRIPTION,Fine Amount
0,1452758967,T753575C,NY,OMT,2019-01-01,71,SDN,TOYOT,P,102,...,Q,BLK,2016,New York,January-2019,2019,01:50PM,Taxi & Limousine Commission,INSP. STICKER-EXPIRED/MISSING,65.0
1,1460097804,JMU9251,NY,PAS,2019-01-01,70,SDN,HONDA,P,113,...,Q,GY,2010,New York,January-2019,2019,07:00AM,Passenger,REG. STICKER-EXPIRED/MISSING,65.0
2,1464592822,31328WG,NY,COM,2019-01-02,14,DELV,FRUEH,P,18,...,NY,WHITE,2004,New York,January-2019,2019,04:32PM,Commercial Vehicles,NO STANDING-DAY/TIME LIMITS,115.0
3,1464592810,63422JM,NY,COM,2019-01-02,14,DELV,INTER,P,18,...,NY,BROWN,1998,New York,January-2019,2019,05:48PM,Commercial Vehicles,NO STANDING-DAY/TIME LIMITS,115.0
4,1464592809,48799JU,NY,COM,2019-01-02,14,VAN,DODGE,P,18,...,NY,WHITE,2005,New York,January-2019,2019,05:55PM,Commercial Vehicles,NO STANDING-DAY/TIME LIMITS,115.0


In [20]:
final_df.to_csv('formatted_dataset.csv', encoding='utf-8', index=False)

In [19]:
tme='10:12A' 
pd.to_datetime(tme).strftime('%I:%M %p')

'10:12 AM'